In [1]:
import warnings
warnings.filterwarnings("ignore")

# installing efficientNet source code
!pip install -U git+https://github.com/qubvel/efficientnet
!pip install tensorflow==2.6.0
!pip install keras~=2.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-4veiejjw
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-4veiejjw
     |████████████████████████████████| 50 kB 4.4 MB/s 
  Created wheel for efficientnet: filename=efficientnet-1.1.1-py3-none-any.whl size=18447 sha256=027755be02008cca4aabba6c017235737b2742d4ebf0ce25f50be27185fbe163
  Stored in directory: /tmp/pip-ephem-wheel-cache-blpazwwe/wheels/11/69/85/814d64d694c96db0eef17b718042d644a1e54f113920481920
Successfully built efficientnet
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564.4 MB 2.5 kB/s 
     |████████████████████████████████| 14.8 MB 11.7 MB/s 
     |████████████████████████████████| 132 kB 70.0 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=3069

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 6.8 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0


In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import efficientnet.keras as enet


In [3]:
# loading Cifar-10 dataset

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
num_classes = 10 
y_train =keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

170508288/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# Swish defination
from keras.backend import sigmoid

class SwishActivation(Activation):
    
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish_act': SwishActivation})

In [25]:
get_custom_objects

<function keras.utils.generic_utils.get_custom_objects>

In [5]:
!pip3 install -U segmentation-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: efficientnet
    Found existing installation: efficientnet 1.1.1
    Uninstalling efficientnet-1.1.1:
      Successfully uninstalled efficientnet-1.1.1


In [6]:
import segmentation_models as sm

sm.set_framework('tf.keras')

sm.framework()

Segmentation Models: using `keras` framework.


'tf.keras'

In [8]:
# Swish defination
from keras.backend import sigmoid
def SwishActivation(x, beta = 1):
    return (x * sigmoid(beta * x))

get_custom_objects().update({'SwishActivation': SwishActivation})


from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation


In [9]:
# loading B0 pre-trained on ImageNet without final aka fiature extractor
import tensorflow.keras as keras
import segmentation_models as sm
import efficientnet.tfkeras as efn

sm.set_framework('tf.keras')

sm.framework()

model = efn.EfficientNetB0(include_top=False, input_shape=(32,32,3), pooling='avg', weights='imagenet')

# building 2 fully connected layer 
x = model.output

x = BatchNormalization()(x)
x = Dropout(0.7)(x)

x = Dense(512)(x)
x = BatchNormalization()(x)
x = SwishActivation(x)
x = Dropout(0.5)(x)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = SwishActivation(x)

# output layer
predictions = Dense(10, activation="softmax")(x)

model_final = Model(inputs = model.input, outputs = predictions)

model_final.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 16, 16, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 16, 16, 32)   128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 16, 16, 32)   0           stem_bn[0][0]                    
______________________________________________________________________________________________

In [ ]:
# ploting keras model for visualization

from keras.utils.vis_utils import plot_model
plot_model(model_final, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

## 10 Epochs

In [25]:
# model compilation
model_final.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.0001),
              metrics=['accuracy'])

mcp_save = ModelCheckpoint('/content/EnetB0_CIFAR10_TL.h5', save_best_only=True, monitor='val_acc')
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, verbose=1,)

#print("Training....")
model_final.fit(x_train, y_train,
              batch_size=32,
              epochs=10,
              validation_split=0.1,
              callbacks=[mcp_save, reduce_lr],
              shuffle=True,
              verbose=1)

Epoch 1/10
1407/1407 [==============================] - 101s 58ms/step - loss: 1.8641 - accuracy: 0.3423 - val_loss: 1.1793 - val_accuracy: 0.5992


Epoch 2/10
1407/1407 [==============================] - 76s 54ms/step - loss: 1.1727 - accuracy: 0.5968 - val_loss: 0.8547 - val_accuracy: 0.7020


Epoch 3/10
1407/1407 [==============================] - 72s 51ms/step - loss: 0.9423 - accuracy: 0.6847 - val_loss: 0.6943 - val_accuracy: 0.7616


Epoch 4/10
1407/1407 [==============================] - 69s 49ms/step - loss: 0.8148 - accuracy: 0.7263 - val_loss: 0.6272 - val_accuracy: 0.7812


Epoch 5/10
1407/1407 [==============================] - 73s 52ms/step - loss: 0.7148 - accuracy: 0.7611 - val_loss: 0.5853 - val_accuracy: 0.7964


Epoch 6/10
1407/1407 [==============================] - 71s 51ms/step - loss: 0.6488 - accuracy: 0.7838 - val_loss: 0.5595 - val_accuracy: 0.8084


Epoch 7/10
1407/1407 [==============================] - 72s 51ms/step - loss: 0.5760 - accuracy: 0.8062 - val_loss: 0.5359 - val_accuracy: 0.8142


Epoch 8/10
1407/1407 [==============================] - 72s 51ms/step - loss: 0.5179 - accuracy: 0.8252 - val_loss: 0.5111 - val_accuracy: 0.8264


Epoch 9/10
1407/1407 [==============================] - 73s 52ms/step - loss: 0.4748 - accuracy: 0.8411 - val_loss: 0.5058 - val_accuracy: 0.8274


Epoch 10/10
1407/1407 [==============================] - 71s 50ms/step - loss: 0.4273 - accuracy: 0.8568 - val_loss: 0.4927 - val_accuracy: 0.8296


In [10]:
!mkdir -p saved_model
model_final.save('/content/efficient') 
from google.colab import files
files.download('/content/efficient') 

INFO:tensorflow:Assets written to: /content/efficient/assets


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
model.save('/content/my_model.h5') 
from google.colab import files
files.download('/content/my_model.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from keras.models import load_model
#get_custom_objects().update({'swish_act': SwishActivation})
new_model = load_model('/content/Efficient_Cifar10.h5', custom_objects={'SwishActivation': SwishActivation})
new_model.summary()


In [ ]:
!pip install foolbox
!pip install timm
!pip install pytorch

In [17]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models
import torch
import torch.nn as nn
import numpy as np
import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD
import foolbox.attacks as attack
import foolbox.attacks as fa
import foolbox as fb
import argparse 
import timm
#assert timm.__version__ == "0.3.2"
import gc
gc.collect()
torch.cuda.empty_cache()
import matplotlib.pyplot as plt
from functools import partial
from functools import partial as pll
from timm.models.vision_transformer import VisionTransformer, _cfg
from timm.models.registry import register_model
from timm.models.layers import trunc_normal_
from tqdm.notebook import tqdm

In [ ]:
x_test

In [18]:
attack = LinfPGD()

In [22]:
model.save_weights('/content/EnetB0_CIFAR10_TL.h5')
!mkdir -p saved_model
model.save('saved_model/my_model') 
model.save

In [23]:
from google.colab import files
files.download('/content/EnetB0_CIFAR10_TL.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# model compilation
new_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.0001),
              metrics=['accuracy'])
_, acc = new_model.evaluate(x_test, y_test)

In [21]:
print("Test Accuracy: {}%".format(acc*100))

NameError: ignored

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

test_pred = model_final.predict(x_test)

In [ ]:
import numpy as np

ax = sns.heatmap(confusion_matrix(np.argmax(y_test, axis=1),np.argmax(test_pred, axis=1)), cmap="YlGnBu",annot=True,fmt="d")

In [ ]:
adef mbConv_block(input_data, block_arg):
    """Mobile Inverted Residual block along with Squeeze and Excitation block."""
    kernel_size = block_arg.kernel_size
    num_repeat= block_arg.num_repeat
    input_filters= block_arg.input_filters
    output_filters= output_filters.kernel_size
    expand_ratio= block_arg.expand_ratio
    id_skip= block_arg.id_skip
    strides= block_arg.strides
    se_ratio= block_arg.se_ratio
    # expansion phase
    expanded_filters =  input_filters * expand_ratio
    x = Conv2D(expanded_filters, 1,  padding='same',  use_bias=False)(input_data)
    x = BatchNormalization()(x)
    x = Activation(swish_activation)(x)
    # Depthwise convolution phase
    x = DepthwiseConv2D(kernel_size, strides,  padding='same',  use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation(swish_activation)(x)
    # Squeeze and excitation phase
    se = GlobalAveragePooling2D()(x)
    se = Reshape((1, 1, expanded_filters ))(x)
    squeezed_filters = max (1, int(input_filters * se_ratio))
    se = Conv2D(squeezed_filters , 1, activation=swish_activation, padding='same')(se)
    se = Conv2D(expanded_filters, 1, activation=’sigmoid’, padding='same')(se)
    x = multiply([x, se])
    # Output phase
    x = Conv2D(output_filters, 1, padding='same', use_bias=False)
    x = BatchNormalization()(x)
    return x
